# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `dataproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import yfinance as yf
import warnings

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# user written modules
import dataprojectyf as dp


# Read and clean data

Import your data, either through an API or manually, and load it. 

## Explore each data set

In order to be able to **explore the raw data**, you may provide **static** and **interactive plots** to show important developments 

**Interactive plot** :

In [68]:
# a. fetch the data for the house representatives
house_raw = dp.fetch_data(mode="house")

# b. clean the data
house_clean = dp.clean_data(house_raw)

request successful
12 invalid dates dropped
1708 invalid tickers dropped
75 options trades dropped


Explain what you see when moving elements of the interactive plot around. 

# Merge data sets

# Analysis

# Conclusion